In [0]:
import pandas as pd

In [0]:
def supres(low, high, n=28, min_touches=2, stat_likeness_percent=1.5, bounce_percent=5):
    """Support and Resistance Testing
    Identifies support and resistance levels of provided price action data.
    Args:
        n(int): Number of frames to evaluate
        low(pandas.Series): A pandas Series of lows from price action data.
        high(pandas.Series): A pandas Series of highs from price action data.
        min_touches(int): Minimum # of touches for established S&R.
        stat_likeness_percent(int/float): Acceptable margin of error for level.
        bounce_percent(int/float): Percent of price action for established bounce.
    
    Returns:
        sup(float): Established level of support or None (if no level)
        res(float): Established level of resistance or None (if no level)
    """
    import pandas as pd
    import numpy as np

    # Collapse into dataframe
    df = pd.concat([high, low], keys = ['high', 'low'], axis=1)
    df['sup'] = pd.Series(np.zeros(len(low)))
    df['res'] = pd.Series(np.zeros(len(low)))
    df['sup_break'] = pd.Series(np.zeros(len(low)))
    df['sup_break'] = 0
    df['res_break'] = pd.Series(np.zeros(len(high)))
    df['res_break'] = 0
    
    for x in range((n-1)+n, len(df)):
        # Split into defined timeframes for analysis
        tempdf = df[x-n:x+1]
        
        # Setting default values for support and resistance to None
        sup = None
        res = None
        
        # Identifying local high and local low
        maxima = tempdf.high.max()
        minima = tempdf.low.min()
        
        # Calculating distance between max and min (total price movement)
        move_range = maxima - minima
        
        # Calculating bounce distance and allowable margin of error for likeness
        move_allowance = move_range * (stat_likeness_percent / 100)
        bounce_distance = move_range * (bounce_percent / 100)
        
        # Test resistance by iterating through data to check for touches delimited by bounces
        touchdown = 0
        awaiting_bounce = False
        for y in range(0, len(tempdf)):
            if abs(maxima - tempdf.high.iloc[y]) < move_allowance and not awaiting_bounce:
                touchdown = touchdown + 1
                awaiting_bounce = True
            elif abs(maxima - tempdf.high.iloc[y]) > bounce_distance:
                awaiting_bounce = False
        if touchdown >= min_touches:
            res = maxima
        # Test support by iterating through data to check for touches delimited by bounces
        touchdown = 0
        awaiting_bounce = False
        for y in range(0, len(tempdf)):
            if abs(tempdf.low.iloc[y] - minima) < move_allowance and not awaiting_bounce:
                touchdown = touchdown + 1
                awaiting_bounce = True
            elif abs(tempdf.low.iloc[y] - minima) > bounce_distance:
                awaiting_bounce = False
        if touchdown >= min_touches:
            sup = minima
        if sup:
            df['sup'].iloc[x] = sup
        if res:
            df['res'].iloc[x] = res
    res_break_indices = list(df[(np.isnan(df['res']) & ~np.isnan(df.shift(1)['res'])) & (df['high'] > df.shift(1)['res'])].index)
    for index in res_break_indices:
        df['res_break'].at[index] = 1
    sup_break_indices = list(df[(np.isnan(df['sup']) & ~np.isnan(df.shift(1)['sup'])) & (df['low'] < df.shift(1)['sup'])].index)
    for index in sup_break_indices:
        df['sup_break'].at[index] = 1
    ret_df = pd.concat([df['sup'], df['res'], df['sup_break'], df['res_break']], keys = ['sup', 'res', 'sup_break', 'res_break'], axis=1)
    return ret_df

In [0]:
tic = 'SPY'
start = '2017-01-01'
end = '2019-06-18'
url = f"https://hw3nhrdos1.execute-api.us-east-2.amazonaws.com/api/history/{tic}/{start}/{end}"

df = pd.read_json(url)[['Open','Close','High','Low','Volume']]

In [57]:
df.tail()

,Open,Close,High,Low,Volume
2019-06-11,290.99,288.90,291.40,288.18,58641300
2019-06-12,288.64,288.39,289.26,287.82,47096300
2019-06-13,289.40,289.58,289.98,288.62,48945200
2019-06-14,289.26,289.26,289.93,288.41,52324700
2019-06-17,289.52,289.37,290.22,289.18,39059600


In [58]:
%%time
levels = supres(df.Low, df.High)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



CPU times: user 7.15 s, sys: 6.21 ms, total: 7.15 s
Wall time: 7.16 s


In [59]:
%%time
for column in levels.columns:
    df[column] = levels[column]

CPU times: user 88.3 ms, sys: 1.96 ms, total: 90.2 ms
Wall time: 91.5 ms


In [60]:
df.columns

Index(['Open', 'Close', 'High', 'Low', 'Volume', 'sup', 'res', 'sup_break',
       'res_break'],
      dtype='object')

In [61]:
# Where resistance level is not nan
df[~np.isnan(df['res'])][['Close','res']]

,Close,res
2017-05-24,231.61,231.72
2017-06-14,235.10,235.84
2017-06-15,234.65,235.84
2017-06-16,234.70,235.84
2017-11-03,251.24,251.28
2017-12-29,260.74,262.44
2018-03-15,268.69,273.98
2018-05-21,268.17,268.86
2018-05-22,267.42,269.03
2018-05-23,268.16,269.03


In [62]:
# Where previous support level has been broken
df[df['sup_break'] == 1][['Close','sup_break']]

,Close,sup_break
2018-10-10,275.47,1
2019-05-29,278.27,1


In [63]:
# Where previous resistance level has been broken
df[df['res_break'] == 1][['Close','res_break']]

,Close,res_break
2017-05-25,232.72,1
2017-06-19,236.66,1
2017-11-06,251.62,1
2018-01-02,262.60,1
2018-06-04,269.67,1
2018-08-21,282.15,1
2018-09-20,289.29,1
2019-01-30,266.41,1


In [64]:
df.head()

,Open,Close,High,Low,Volume,sup,res,sup_break,res_break
2017-01-03,215.68,215.88,216.44,214.57,91366500,NaN,NaN,0,0
2017-01-04,216.24,217.16,217.32,216.23,78744400,NaN,NaN,0,0
2017-01-05,216.86,216.99,217.16,216.11,78379000,NaN,NaN,0,0
2017-01-06,217.11,217.76,218.28,216.51,71559900,NaN,NaN,0,0
2017-01-09,217.48,217.04,217.63,217.01,46265300,NaN,NaN,0,0


In [65]:
# Round numbers within 10% of the closing price
len([i for i in df['Close'] if (i % 2 <= 0.1 or i % 3 <= 0.1)])

35

In [0]:
df['around_round'] = [1 if (i % 2 <= 0.1 or i % 3 <= 0.1) else 0 for i in df['Close']]

In [68]:
df.around_round.value_counts()

0    582
1     35
Name: around_round, dtype: int64

In [71]:
df.sample(5)

,Open,Close,High,Low,Volume,sup,res,sup_break,res_break,around_round
2019-03-27,281.11,279.65,281.76,277.93,72224700,NaN,NaN,0,0,0
2017-01-04,216.24,217.16,217.32,216.23,78744400,NaN,NaN,0,0,0
2018-02-08,261.86,251.72,262.02,251.68,246449500,NaN,NaN,0,0,0
2018-11-09,276.19,274.94,276.40,273.37,98812600,NaN,NaN,0,0,0
2019-01-07,251.58,253.26,254.83,250.59,103139100,NaN,NaN,0,0,0


In [0]:
# Investability index


In [0]:
# trace = go.Candlestick(x=df.index,
#                 open=df.Open,
#                 high=df.High,
#                 low=df.Low,
#                 close=df.Close)
# data = [trace]
# layout = {
#     'title': 'The Great Recession',
#     'yaxis': {'title': 'AAPL Stock'},
#     'xaxis': {'rangeslider':{'visible': False}},
#     'shapes': [{
#         'x0': '2017-12-09', 'x1': '2017-12-09',
#         'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
#         'line': {'color': 'rgb(30,30,30)', 'width': 1}
#     }],
#     'annotations': [{
#         'x': '2017-12-09', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
#         'showarrow': False, 'xanchor': 'left',
#         'text': 'Increase Period Begins'
#     }]
# }
# fig = dict(data=data, layout=layout)
# py.iplot(fig, filename='aapl-recession-candlestick')